# OneClassSVM
https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Read and standarize the dataset
df = pd.read_csv('data/selected_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
df = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns.values)
df['target'] = y

# Divide dataframe depending on target
df_correct = df.loc[df['target'] == 1]
df_incorrect = df.loc[df['target'] == 0]

# Print each target shape
print('Target 1 shape: ' + str(df_correct.shape))
print('Target 0 shape: ' + str(df_incorrect.shape))

Target 1 shape: (5055, 39)
Target 0 shape: (4959, 39)


In [2]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

df_correct = shuffle(df_correct).reset_index(drop=True)
df_train, df_test = train_test_split(df_correct, test_size=0.2, random_state=0)
df_test = df_test.append(df_incorrect).reset_index(drop=True)

X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]

In [3]:
# -1 = outlier
#  1 = inlier
from sklearn.metrics import f1_score

def get_f1_score(y_pred):
    y_true = y_test.copy()
    y_true[y_true == 0] = -1
    print()
    print("f1_score: ", f1_score(y_true, y_pred))    

### rbf

In [4]:
import collections
from sklearn.svm import OneClassSVM

rbf = OneClassSVM(kernel='rbf', gamma='scale').fit(X_train)
pred_train = rbf.predict(X_train)
pred_test = rbf.predict(X_test)

print("PRED TRAIN", collections.Counter(pred_train))
print("PRED TEST ", collections.Counter(pred_test))
get_f1_score(pred_test)

PRED TRAIN Counter({-1: 2023, 1: 2021})
PRED TEST  Counter({-1: 4298, 1: 1672})

f1_score:  0.39731643682445034


### poly

In [5]:
import collections
from sklearn.svm import OneClassSVM

rbf = OneClassSVM(kernel='poly', gamma='scale').fit(X_train)
pred_train = rbf.predict(X_train)
pred_test = rbf.predict(X_test)

print("PRED TRAIN", collections.Counter(pred_train))
print("PRED TEST ", collections.Counter(pred_test))
get_f1_score(pred_test)

PRED TRAIN Counter({-1: 2028, 1: 2016})
PRED TEST  Counter({-1: 3839, 1: 2131})

f1_score:  0.29471674092934436


### linear

In [6]:
import collections
from sklearn.svm import OneClassSVM

rbf = OneClassSVM(kernel='linear', gamma='scale').fit(X_train)
pred_train = rbf.predict(X_train)
pred_test = rbf.predict(X_test)

print("PRED TRAIN", collections.Counter(pred_train))
print("PRED TEST ", collections.Counter(pred_test))
get_f1_score(pred_test)

PRED TRAIN Counter({1: 2023, -1: 2021})
PRED TEST  Counter({-1: 4798, 1: 1172})

f1_score:  0.4525881814017407


### sigmoid

In [7]:
import collections
from sklearn.svm import OneClassSVM

rbf = OneClassSVM(kernel='sigmoid', gamma='scale').fit(X_train)
pred_train = rbf.predict(X_train)
pred_test = rbf.predict(X_test)

print("PRED TRAIN", collections.Counter(pred_train))
print("PRED TEST ", collections.Counter(pred_test))
get_f1_score(pred_test)

PRED TRAIN Counter({1: 2022, -1: 2022})
PRED TEST  Counter({-1: 4523, 1: 1447})

f1_score:  0.42229454841334424
